In [ ]:
import json
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from Helper_Functions import *
from tqdm import tqdm
import time
from scipy.optimize import linear_sum_assignment
import warnings
import joblib
from scipy.ndimage import gaussian_filter
warnings.filterwarnings("ignore")

### Load Data:

In [ ]:
a_outfielders,a_gk,_ = Load_Test_Data('Sample_Game_1_RawTrackingData_Away_Team.csv',half=2)
h_outfielders,h_gk,ball_pos = Load_Test_Data('Sample_Game_1_RawTrackingData_Home_Team.csv',half=2)

### Load Models:

In [ ]:
model_x = joblib.load('Model_x.pkl')
model_y = joblib.load('Model_y.pkl')
results_x = model_x.fit()
results_y = model_y.fit()

### Parameters:

In [ ]:
radius = 30
frames_per = 25

### Test Creator:

In [ ]:
def Filter_Positions(frame):
    b_pos = ball_pos[frame]
    tlocs = h_outfielders[frame]
    olocs = a_outfielders[frame]
    tkeep = h_gk[frame]
    okeep = a_gk[frame]
    
    
    tkept = np.zeros(20)
    tgkkept = np.zeros(2)
    for m in range(10):
        distance= np.sqrt((tlocs[2*m] - b_pos[0])**2 + (tlocs[2*m + 1] - b_pos[1])**2)
        tkept[2*m:2*(m+1)]  = distance < radius
    tgkkept[:] = np.sqrt((tkeep[0] - b_pos[0])**2 + (tkeep[1] - b_pos[1])**2) < radius
    tlocs = tlocs[tkept==1]
    tkeep = tkeep[tgkkept==1] 
    
    okept = np.zeros(20)
    ogkkept = np.zeros(2)
    for m in range(10):
        distance = np.sqrt((olocs[2*m] - b_pos[0])**2 + (olocs[2*m + 1] - b_pos[1])**2)
        okept[2*m:2*(m+1)] = distance < radius
    ogkkept[:] = np.sqrt((okeep[0] - b_pos[0])**2 + (okeep[1] - b_pos[1])**2) < radius
    olocs = olocs[okept==1]
    okeep = okeep[ogkkept==1]
    
    tlocs = tlocs.reshape((int(len(tlocs)/2),2))
    tkeep = tkeep.reshape((int(len(tkeep)/2),2))
    olocs = olocs.reshape((int(len(olocs)/2),2))
    okeep = okeep.reshape((int(len(okeep)/2),2))
    return tlocs,tkeep,olocs,okeep,b_pos

### Fitting

In [ ]:
def Ndist(x,mu,sigma):

    return np.sum(-(np.square(x-mu))/(2*(sigma**2)) - np.log((sigma**2)))



def Predict(all_positions,all_times,curr_frame,all_exog):
    if len(all_positions) > 100:
        all_positions = all_positions[-100:]
        all_times = all_times[-100:]
        curr = 100
        all_exog = all_exog[curr_frame-100:]
    else:
        curr = curr_frame
    models = [results_x,results_y]
    outputs = np.zeros((20,2,2))
    for m in range(20):
        for k in range(2):
            p = all_positions[:curr,m,k]
            t = all_times[:curr,m]
            exog = all_exog[:curr,k]
            p_smoothed = np.zeros(curr)
            t[t==-10] = 0
            discretes = np.arange(len(t)).astype(int)
            t = np.append(-50,t)
            p_filt= p[t[1:] != t[:-1]]
            
            
            
            t_filt = t[1:][t[1:] != t[:-1]]
            d_filt =discretes[t[1:] != t[:-1]]
            
            if len(p) > 1:
                index = np.max(np.arange(len(p_smoothed))[t[1:] != t[:-1]])
                
                
                if curr - index > 1:
                
                    for n in range(len(p_filt)-1):
                        p_smoothed[int(d_filt[n]):int(d_filt[n+1])] = np.linspace(p_filt[n],p_filt[n+1],int(d_filt[n+1] - d_filt[n]))
                    p_smoothed[int(d_filt[len(p_filt)-1])] = p_filt[len(p_filt)-1]
                    if len(p_smoothed) > 1 and index > 1 and len(np.unique(exog[:index+1])) > 2:
                        pred_results = models[k].apply(p_smoothed[:index+1],exog=exog[:index+1])

                        preds = pred_results.get_prediction(start=index, end=curr,exog = all_exog[index+1:curr+1,k])

                        outputs[m,k,0] = preds.predicted_mean[-1]
                        outputs[m,k,1] = preds.se_mean[-1]
                        #if curr - index == 1:
                        #    print(p_smoothed[:index+1])
                        #    print(p_filt[-1])
                        #    print(curr-index)
                        #    print(outputs[m,k,0])
                        #    print('_____________________________________________')
                    else:
                        outputs[m,k,0] = p[-1]
                        outputs[m,k,1] = (curr-index)*4
                else:
                    outputs[m,k,0] = p[-1]
                    outputs[m,k,1] = (curr-index)*4
            else:

                outputs[m,k,0] = p[-1]
                outputs[m,k,1] = (curr_frame+1)*4
    return outputs


initialise = True




curr_positions = np.zeros((22,2))
curr_times = (-10)*np.ones(22)

all_positions = np.zeros((15000,22,2))
all_times = np.zeros((15000,22))
true_times = np.zeros((15000,22))
visible = np.zeros((15000,22))
all_exog = np.zeros((15000,2))
for dat_num in range(min(15000,int(np.around(len(h_outfielders)/frames_per))-2)):
    frame = frames_per*dat_num
    tlocs,tkeep,olocs,okeep,b_pos = Filter_Positions(frame)
    all_exog[dat_num] = b_pos
all_exog = gaussian_filter(all_exog,sigma=4)
for dat_num in tqdm(range(min(15000,int(np.around(len(h_outfielders)/frames_per))-2))):
    frame = frames_per*dat_num
    t =dat_num*frames_per/25
    
    tlocs,tkeep,olocs,okeep,b_pos = Filter_Positions(frame)
    
    if initialise:
        for n,loc in enumerate(tlocs):
            curr_positions[n] = loc
            curr_times[n] = 0
            visible[dat_num,n] = 1
        other_ys = np.linspace(0,80,(10-n+2))
        for m in range(n,10):
            curr_positions[m] =[30,other_ys[m-n+1]]
            curr_times[m] = - 10


        for n,loc in enumerate(olocs):
            curr_positions[n+10] = loc
            curr_times[n+10] =  0
            visible[dat_num,n+10] = 1
        other_ys = np.linspace(0,80,(10-n+2))
        for m in range(n,10):
            curr_positions[m+10] =[90,other_ys[m-n+1]]
            curr_times[m+10] =  - 10



        if len(tkeep) > 0:
            curr_positions[20] = tkeep
            visible[dat_num,20] = 1
        else:
            curr_positions[20] = [10,40]

        if len(okeep) > 0:
            curr_positions[21] = okeep
            visible[dat_num,21] = 1
        else:
            curr_positions[21] = [110,40]
        initialise = False
        all_positions[dat_num] = curr_positions
        all_times[dat_num] = curr_times
        true_times[dat_num] = 0
        dat_num += 1
        

        continue




    outputs = Predict(all_positions[:dat_num],all_times[:dat_num],dat_num,all_exog)

    dists = np.zeros((10,len(tlocs)))
    for n in range(10):
        for m in range(len(tlocs)):
            dists[n,m] = Ndist(tlocs[m],outputs[n,:,0],outputs[n,:,1])
    row_indices, col_indices = linear_sum_assignment(-dists)
    for i, j in zip(row_indices, col_indices):
        curr_positions[i] = tlocs[j]
        curr_times[i] = t
        visible[dat_num,i] = 1


    dists = np.zeros((10,len(olocs)))
    for n in range(10):
        for m in range(len(olocs)):
            dists[n,m] = Ndist(olocs[m],outputs[n+10,:,0],outputs[n+10,:,1])
    row_indices, col_indices = linear_sum_assignment(-dists)
    for i, j in zip(row_indices, col_indices):
        curr_positions[i+10] = olocs[j]
        curr_times[i+10] = t
        visible[dat_num,i+10] = 1

    if len(tkeep) > 0:


        curr_positions[20] =tkeep
        curr_times[20] = t
        visible[dat_num,20] = 1

    if len(okeep) > 0:
       
        curr_positions[21] = okeep
        curr_times[21] = t
        visible[dat_num,21] = 1


    

    all_positions[dat_num] = curr_positions
    
    all_times[dat_num] = curr_times

    true_times[dat_num] = t
    



all_positions = all_positions[:dat_num]
all_times = all_times[:dat_num]
visible = visible[:dat_num]
true_times = true_times[:dat_num]
all_exog = all_exog[:dat_num]

In [ ]:
len(np.unique(all_exog[dat_num-100:dat_num]))

### Create Smoothed Trajectories

In [ ]:
mu = 0.6
velocities = np.zeros((dat_num-1,2))
for n in range(dat_num-1):
    allowed = (visible[n] == 1)&(visible[n+1] == 1)&(np.arange(22)<20)
    if np.sum(allowed) > 0:
        velocities[n] = np.mean(all_positions[n+1,allowed] - all_positions[n,allowed],0)*mu

filtered_positions = np.copy(all_positions)
filtered_positions_no_velocity = np.copy(all_positions)
filtered_positions_static = np.copy(all_positions)
for n in range(22):
    player_vis = visible[:,n] == 1
    vis_indices = np.arange(dat_num)[player_vis]
    vis_indices = np.append(vis_indices,dat_num-1)
    for old,new in zip(vis_indices[:-1],vis_indices[1:]):
        
        tstart = all_times[old,n]
        tend = all_times[new,n]
    
        pstart = all_positions[old,n]
        pend = all_positions[new,n]
        if tend == tstart:
            continue
        if n < 20:
            tot_veloc = np.sum(velocities[old:new-1],0)
            veloc_shifts = np.cumsum(velocities[old:new-1],0)
            veloc_shifts = np.append([[0,0]],veloc_shifts,axis=0)

            

            filtered_positions[old:new,n,0] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[0]-pstart[0]-tot_veloc[0]) + pstart[0] + veloc_shifts[:,0]
            filtered_positions[old:new,n,1] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[1]-pstart[1]-tot_veloc[1]) + pstart[1] + veloc_shifts[:,1]
            #filtered_positions[old:new,n,1] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[1]-pstart[1]) + pstart[1]
        else:
            filtered_positions[old:new,n,0] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[0]-pstart[0]) + pstart[0]
            filtered_positions[old:new,n,1] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[1]-pstart[1]) + pstart[1]
            
        
        filtered_positions_no_velocity[old:new,n,0] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[0]-pstart[0]) + pstart[0]
        filtered_positions_no_velocity[old:new,n,1] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[1]-pstart[1]) + pstart[1]

        filtered_positions_static[old:new,n,0] =  pstart[0]
        filtered_positions_static[old:new,n,1] = pstart[1]
        

### Varying Velocity Weight

We investigate the effect of varying the velocity weight

In [ ]:
processed_positions = []
mu_range = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
for mu in mu_range:
    velocities = np.zeros((dat_num-1,2))
    for n in range(dat_num-1):
        allowed = (visible[n] == 1)&(visible[n+1] == 1)&(np.arange(22)<20)
        if np.sum(allowed) > 0:
            velocities[n] = np.mean(all_positions[n+1,allowed] - all_positions[n,allowed],0)*mu
    filtered_positions_mu = np.copy(all_positions)
    for n in range(22):
        player_vis = visible[:,n] == 1
        vis_indices = np.arange(dat_num)[player_vis]
        vis_indices = np.append(vis_indices,dat_num-1)
        for old,new in zip(vis_indices[:-1],vis_indices[1:]):
            
            tstart = all_times[old,n]
            tend = all_times[new,n]
        
            pstart = all_positions[old,n]
            pend = all_positions[new,n]
            if tend == tstart:
                continue
            if n < 20:
                tot_veloc = np.sum(velocities[old:new-1],0)
                veloc_shifts = np.cumsum(velocities[old:new-1],0)
                
                veloc_shifts = np.append([[0,0]],veloc_shifts,axis=0)

                

                filtered_positions_mu[old:new,n,0] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[0]-pstart[0]-tot_veloc[0]) + pstart[0] + veloc_shifts[:,0]
                filtered_positions_mu[old:new,n,1] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[1]-pstart[1]-tot_veloc[1]) + pstart[1] + veloc_shifts[:,1]
            else:
                filtered_positions_mu[old:new,n,0] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[0]-pstart[0]) + pstart[0]
                filtered_positions_mu[old:new,n,1] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[1]-pstart[1]) + pstart[1]
    processed_positions.append(filtered_positions_mu)
            

### Error Computation (In Phase)

In [ ]:
def compute_in_phase_errors(filtered_positions:np.ndarray, file_path:str):
    """
    Compute the errors for a set of positions 
    """
    delays = np.zeros((len(filtered_positions),20))
    errors = np.zeros(len(filtered_positions))
    off_camera = np.zeros((len(filtered_positions)))
    all_errors = np.zeros((len(filtered_positions),20))
    for nn in range(30,len(filtered_positions)-30):
        delays[nn] = nn-all_times[nn,:20]
        
        h_guess = filtered_positions[nn,:10]
        a_guess = filtered_positions[nn,10:20]
        
        h_true = h_outfielders[nn*frames_per].reshape((10,2))
        a_true =  a_outfielders[nn*frames_per].reshape((10,2))
        
        dists_h = np.zeros((10,10))
        for n in range(10):
            for m in range(10):
                dists_h[n,m] =np.sum(np.square(h_true[n]-h_guess[m]))
        row_indices, col_indices = linear_sum_assignment(dists_h)
        tot = np.sum([dists_h[r,c] for r,c in zip(row_indices,col_indices)])
        all_errors[nn,:10] = [dists_h[r,c] for r,c in zip(row_indices,col_indices)]
        ptot = np.sum([dists_h[r,c]>0 for r,c in zip(row_indices,col_indices)])
        
        dists_a = np.zeros((10,10))
        for n in range(10):
            for m in range(10):
                dists_a[n,m] =np.sum(np.square(a_true[n]-a_guess[m]))
        row_indices, col_indices = linear_sum_assignment(dists_a)
        tot += np.sum([dists_a[r,c] for r,c in zip(row_indices,col_indices)])
        ptot += np.sum([dists_a[r,c]>0 for r,c in zip(row_indices,col_indices)])
        errors[nn] = tot/(1e-8+ptot)
        all_errors[nn,10:] = [dists_a[r,c] for r,c in zip(row_indices,col_indices)]
        off_camera[nn] = ptot
        
    real_est = all_errors[all_errors > 0]
    real_del = delays[all_errors > 0]

    print('Percentage within 5m:')
    print(str(np.around(100*np.sum(real_est < 25)/len(real_est),2)) +'%')
    print(' ')
    print('Percentage within 10m:')
    print(str(np.around(100*np.sum(real_est < 100)/len(real_est),2)) +'%')
    print(' ')
    print('RMSE:')
    print(str(np.around(np.sqrt(np.mean(real_est)),2)) + 'm')
    print(' ')
    print('MAE:')
    print(str(np.around(np.mean(np.sqrt(real_est)),2)) + 'm')
    np.save(file_path,all_errors)

compute_in_phase_errors(filtered_positions, '2H.npy')
compute_in_phase_errors(filtered_positions_no_velocity, '2H_no_velocity.npy')
compute_in_phase_errors(filtered_positions_static, '2H_static.npy')
for index, positions in enumerate(processed_positions):
    print('______________')
    print(f'Results for mu = {mu_range[index]}')
    compute_in_phase_errors(positions, f'2H_mu_{mu_range[index]}.npy')

### Error Computation (Out of Phase)

In [ ]:
def compute_out_of_phase_errors(filtered_positions, file_path):
    """
    Compute the out of phase errors for different positions
    """
    delays = np.zeros((len(filtered_positions),20))
    errors = np.zeros(len(filtered_positions))
    off_camera = np.zeros((len(filtered_positions)))
    all_errors = np.zeros((len(filtered_positions),20))
    for nn in range(30,len(filtered_positions)-30):
        delays[nn] = nn-all_times[nn,:20]
        
        h_guess = 0.5*(filtered_positions[nn,:10] + filtered_positions[nn+1,:10] )
        a_guess = 0.5*(filtered_positions[nn,10:20] + filtered_positions[nn+1,10:20])
        
        h_true = h_outfielders[nn*frames_per + int(0.5*frames_per)].reshape((10,2))
        a_true =  a_outfielders[nn*frames_per+ int(0.5*frames_per)].reshape((10,2))
        
        dists_h = np.zeros((10,10))
        for n in range(10):
            for m in range(10):
                dists_h[n,m] =np.sum(np.square(h_true[n]-h_guess[m]))
        row_indices, col_indices = linear_sum_assignment(dists_h)
        tot = np.sum([dists_h[r,c] for r,c in zip(row_indices,col_indices)])
        all_errors[nn,:10] = [dists_h[r,c] for r,c in zip(row_indices,col_indices)]
        ptot = np.sum([dists_h[r,c]>0 for r,c in zip(row_indices,col_indices)])
        
        dists_a = np.zeros((10,10))
        for n in range(10):
            for m in range(10):
                dists_a[n,m] =np.sum(np.square(a_true[n]-a_guess[m]))
        row_indices, col_indices = linear_sum_assignment(dists_a)
        tot += np.sum([dists_a[r,c] for r,c in zip(row_indices,col_indices)])
        ptot += np.sum([dists_a[r,c]>0 for r,c in zip(row_indices,col_indices)])
        errors[nn] = tot/(1e-8+ptot)
        all_errors[nn,10:] = [dists_a[r,c] for r,c in zip(row_indices,col_indices)]
        off_camera[nn] = ptot
        
    real_est = all_errors[all_errors > 0]
    real_del = delays[all_errors > 0]

    print('Percentage within 5m:')
    print(str(np.around(100*np.sum(real_est < 25)/len(real_est),2)) +'%')
    print(' ')
    print('Percentage within 10m:')
    print(str(np.around(100*np.sum(real_est < 100)/len(real_est),2)) +'%')
    print(' ')
    print('RMSE:')
    print(str(np.around(np.sqrt(np.mean(real_est)),2)) + 'm')
    print(' ')
    print('MAE:')
    print(str(np.around(np.mean(np.sqrt(real_est)),2)) + 'm')
    np.save(file_path,all_errors)
compute_out_of_phase_errors(filtered_positions, '2H_O.npy')
compute_out_of_phase_errors(filtered_positions_no_velocity,'2H_O_no_velocity.npy')
compute_in_phase_errors(filtered_positions_static, '2H_O_static.npy')
for index, positions in enumerate(processed_positions):
    print('______________')
    print(f'Results for mu = {mu_range[index]}')
    compute_out_of_phase_errors(positions, f'2H_O_mu_{mu_range[index]}.npy')

In [ ]:
len(all_positions)

In [ ]:
np.mean(all_errors == 0)*20

In [ ]:
for index in range(0,30):
    img = plt.imread('Football Pitch.jpg')
    plt.figure(figsize = (12,8))
    plt.imshow(img)
    plt.scatter(10*filtered_positions[index,:10,0],10*filtered_positions[index,:10,1],marker='o',label = 'Off-camera predictions (T1)',s=300,color = (1,0.5,0.5,1))
    plt.scatter(10*filtered_positions[index,:10,0][visible[index,:10] == 1],10*all_positions[index,:10,1][visible[index,:10] == 1],marker='o',label = 'On-camera (T1)',s=300,color = (1,0,0,1))

    plt.scatter(10*filtered_positions[index,10:20,0],10*filtered_positions[index,10:20,1],marker='o',label = 'Off-camera predictions (T2)',s=300,color=(0.5,0.5,1,1))
    plt.scatter(10*filtered_positions[index,10:20,0][visible[index,10:20] == 1],10*all_positions[index,10:20,1][visible[index,10:20] == 1],marker='o',label = 'On-camera (T2)',s=300,color=(0,0,1,1))

    plt.scatter(10*filtered_positions[index,20:,0],10*filtered_positions[index,20:,1],marker='o',label = 'Off-camera predictions (GK)',s=300,color = (0.5,0.5,0.5,1))
    plt.scatter(10*filtered_positions[index,20:,0][visible[index,20:] == 1],10*filtered_positions[index,20:,1][visible[index,20:] == 1],marker='o',label = 'On-camera (GK)',s=300,color=(0,0,0,1))
    
    plt.scatter(10*h_outfielders[index*frames_per].reshape((10,2))[:,0],10*h_outfielders[index*frames_per].reshape((10,2))[:,1],label = 'Truth (T1)',color = (1,1,0),marker = 'x',s=100)
    plt.scatter(10*a_outfielders[index*frames_per].reshape((10,2))[:,0],10*a_outfielders[index*frames_per].reshape((10,2))[:,1],label = 'Truth (T2)',color = (0,1,1),marker = 'x',s=100)
    plt.xticks([])
    plt.yticks([])
    plt.legend()
    plt.show()
    print(np.sqrt(all_errors[index]))